In [1]:
import copy
import numpy as np
import os
import pandas as pd
pd.options.display.max_rows = 20
pd.get_option("display.max_columns")

20

## 使い方
- pathを変更する

In [2]:
paths = ['tfidf_text/0thLayer','tfidf_text/1stLayer','tfidf_text/2ndLayer']

In [3]:
all_nodes = set()
link_dict = dict()
link_df = pd.DataFrame(columns=['from','to','value'])
for path in paths:
    files = os.listdir(path)
    for file in files:
        query,_ = os.path.splitext(file)
        if not query.startswith('.'):
            all_nodes.add(query)
            link_dict[query] = []
            records = []
            file = open('{0}/{1}.txt'.format(path,query),'r')
            lines = file.readlines()
            for i in range(20):
                to_query = lines[i].split(',')[0][2:-1]
                value = float(lines[i].split(',')[1][1:-2])
                if not query == to_query:  #セルフノード削除
                    all_nodes.add(to_query)
                    link_dict[query].append(to_query)
                    records.append([query, to_query, value])
            df_ = pd.DataFrame(records,columns=link_df.columns)
            link_df = link_df.append(df_)

# link_df.drop(link_df[link_df["to"]==link_df["from"]].index) 
link_df = link_df.reset_index(drop=True)

In [4]:
#link_df.to_csv('links.csv',encoding='utf-8',index=False)

In [5]:
#link_df = pd.read_csv('links.csv')

初期化

In [6]:
# link_df[link_df['from'] == '医療']

In [7]:
len(link_dict)

570

In [8]:
len(all_nodes)

2798

In [9]:
len(link_df)

11392

In [10]:
tmp = pd.DataFrame()
tmp['from'] = link_df['to']
tmp['to'] = link_df['from']
tmp['value'] = link_df['value']
link_df = link_df.append(tmp)
link_df.reset_index(drop=True)

new_link_dict = copy.deepcopy(link_dict)
for key,value in link_dict.items():
    for val in value:
        if val in link_dict.keys():
            new_link_dict[val].append(key)
        else:
            new_link_dict[val] = [key]
link_dict = new_link_dict

In [11]:
univs = {
    "東京大学":"from_ut",
    "京都大学":"from_ky",
    "早稲田大学":"from_wa",
    "慶應義塾大学":"from_ko",
    "明治大学":"from_me"
}

df版

In [12]:
# node_df = pd.DataFrame(
#     index=range(len(all_nodes)), 
#     columns=[
#         'query',
#         'from_ut',
#         'from_ky',
#         'from_wa',
#         'from_ko',
#         'from_me',
# #         'label',
# #         'shortest_step'
#     ])
# node_df['query'] = list(all_nodes)
# node_df = node_df.fillna(float(99))

In [13]:
# for key, value in univs.items():
#     node_df.loc[node_df['query'] == key, value] = 0

In [14]:
# from_querys = univs.keys()

In [15]:
# next_step = link_df[link_df['from'].isin(from_querys)]

In [16]:
# for from_query in from_querys:
#     value = node_df.loc[node_df['query'] == from_query,univs.values()]
#     for to_query in link_dict[from_query]:
#         for column in univs.values():
#             node_df.loc[node_df['query'] == to_query, column] = value

dict版

In [17]:
def check_layer(from_querys,node_dict,link_dict,univs):
    for from_query in from_querys:
        distance_dict = copy.deepcopy(node_dict[from_query])
        for key in distance_dict.keys():
            distance_dict[key] += 1
        try:
            for to_query in link_dict[from_query]:
                for key in univs.values():
                    node_dict[to_query][key] = min(distance_dict[key],node_dict[to_query][key])
        except KeyError:
            pass
    
    return node_dict

In [18]:
def get_next_from_querys(link_df,from_querys):
    next_step = link_df[link_df['from'].isin(from_querys)]
    
    return list(next_step['to'])

In [19]:
node_dict = {}
for query in all_nodes:
    node_dict[query] = {
    "from_ut":99,
    "from_ky":99,
    "from_wa":99,
    "from_ko":99,
    "from_me":99
}

#初期化
node_dict["東京大学"]["from_ut"] = 0
node_dict["京都大学"]["from_ky"] = 0
node_dict["早稲田大学"]["from_wa"] = 0
node_dict["慶應義塾大学"]["from_ko"] = 0
node_dict["明治大学"]["from_me"] = 0

In [20]:
#1層目
from_querys = list(univs.keys())

In [21]:
for x in range(30):
    node_dict = check_layer(from_querys,node_dict,link_dict,univs)
    from_querys = get_next_from_querys(link_df,from_querys)

In [22]:
list_of_list = []
for key, value in node_dict.items():
    list_of_list.append([key,value['from_ut'],value['from_ky'],value['from_wa'],value['from_ko'],value['from_me']])
node_df = pd.DataFrame(list_of_list,columns=['query','from_ut','from_ky','from_wa','from_ko','from_me'])

In [23]:
node_df['shortest_step'] = np.min(node_df[list(univs.values())],axis=1)

In [24]:
# for column in ['ut','ky','wa','ko','me']:
#     node_df['label_{}'.format(column)] = node_df['from_{}'.format(column)] == node_df['shortest_step']

In [25]:
univ_df = pd.DataFrame()
univ_df = univ_df.append(node_df[node_df['query'] == '東京大学']).append(node_df[node_df['query'] == '京都大学'].append(node_df[node_df['query'] == '明治大学'])).append(node_df[node_df['query'] == '早稲田大学']).append(node_df[node_df['query'] == '慶應義塾大学'])

In [26]:
univ_df

,query,from_ut,from_ky,from_wa,from_ko,from_me,shortest_step,label_ut,label_ky,label_wa,label_ko,label_me
1372,東京大学,0,2,2,2,2,0,True,False,False,False,False
2436,京都大学,2,0,2,2,2,0,False,True,False,False,False
820,明治大学,2,2,2,2,0,0,False,False,False,False,True
2408,早稲田大学,2,2,0,2,2,0,False,False,True,False,False
2628,慶應義塾大学,2,2,2,0,2,0,False,False,False,True,False


In [27]:
node_df = node_df.drop(univ_df.index)

In [28]:
node_df = univ_df.append(node_df).reset_index(drop=True)

In [29]:
node_df['index'] = node_df.index

In [30]:
node_df

,query,from_ut,from_ky,from_wa,from_ko,from_me,shortest_step,label_ut,label_ky,label_wa,label_ko,label_me,index
0,東京大学,0,2,2,2,2,0,True,False,False,False,False,0
1,京都大学,2,0,2,2,2,0,False,True,False,False,False,1
2,明治大学,2,2,2,2,0,0,False,False,False,False,True,2
3,早稲田大学,2,2,0,2,2,0,False,False,True,False,False,3
4,慶應義塾大学,2,2,2,0,2,0,False,False,False,True,False,4
5,国勢,3,3,3,3,3,3,True,True,True,True,True,5
6,開催,2,2,2,2,2,2,True,True,True,True,True,6
7,品質,4,2,3,4,4,2,False,True,False,False,False,7
8,レポート,4,3,4,4,4,3,False,True,False,False,False,8
9,インストール,4,4,4,4,3,3,False,False,False,False,True,9


In [31]:
def get_node_index(x):
    number = node_df[node_df['query']==x]['index']
    return int(number)

In [32]:
link_df['from_id'] = link_df['from'].map(get_node_index)
link_df['to_id'] = link_df['to'].map(get_node_index)

In [33]:
link_df

,from,to,value,from_id,to_id
0,慶應義塾大学,研究,0.025952,4,2088
1,慶應義塾大学,学部,0.022282,4,1533
2,慶應義塾大学,月,0.020959,4,2657
3,慶應義塾大学,学生,0.020782,4,1924
4,慶應義塾大学,情報,0.019950,4,1928
5,慶應義塾大学,入試,0.018877,4,1725
6,慶應義塾大学,医学部,0.018081,4,48
7,慶應義塾大学,紹介,0.016290,4,2040
8,慶應義塾大学,キャンパス,0.016262,4,2521
9,慶應義塾大学,マンション,0.015036,4,777


In [111]:
# node_df.to_csv('nodes.csv')
# link_df.to_csv('links.csv')

In [148]:
def set_label(x):
    label = []
    univ = ['ut','ky','me','wa','ko']
    for i in range(len(univ)):
        if x['label_{}'.format(univ[i])]:
            label.append(i)
    return label

In [149]:
node_df['label'] = node_df.apply(set_label,axis=1)

In [150]:
labeled_df = node_df.loc[:,['index','label']]

In [151]:
labeled_df

,index,label
0,0,[0]
1,1,[1]
2,2,[2]
3,3,[3]
4,4,[4]
5,5,"[0, 1, 2, 3, 4]"
6,6,"[0, 1, 2, 3, 4]"
7,7,[1]
8,8,[1]
9,9,[2]


In [152]:
# labeled_df.to_csv('label.csv')